## 4 boites en EU :
  
  - box1 : 75 256 98 326
  - box2 : 70 253 324 568
  - box3 : 60 245 563 826
  - box4 : 21 208 1089 1391


In [1]:
import sys
import xarray as xr
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import glob
from netCDF4 import Dataset
import cartopy.crs as ccrs
import os

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
%autosave 60

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


Autosaving every 60 seconds


In [2]:
sys.path.insert(0,"/scratch/cnt0024/hmg2840/albert7a/DEV/git/powerspec/powerspec")
import powerspec as pp

In [3]:
sys.path.insert(0,"/scratch/cnt0024/hmg2840/albert7a/DEV/git/diags-CMEMS-on-occigen/common-lib/")
import GriddedData
import WavenumberSpectrum as ws


In [4]:
# Some functions that allow us to compute spectra in boxes

##########################################
def get_values_in_box(imin,imax,jmin,jmax,data):
    values = data[:,0,jmin:jmax+1,imin:imax+1]
    values = ma.masked_invalid(values)
    return values


##########################################
def compute_spec_for_box(imin,imax,jmin,jmax,data,Mth,navlon,navlat,name,WaveSpecResult,param):
    var = get_values_in_box(imin,imax,jmin,jmax,data)
    pspec,kspec = compute_spec(var,navlon,navlat)
    np.savez(WaveSpecResult+'WaveSpec_'+name+'_'+param+'_'+Mth, kspec=kspec ,pspec=pspec)

##########################################
def compute_spec(data,navlon,navlat):
    days = len(data)
    mth_pspec = []
    for it in np.arange(0,days):
        arr = data[it]
        datab = arr.squeeze()
        pspec,kstep = pp.wavenumber_spectra(datab,navlon,navlat)
        mth_pspec.append(pspec)
    mthly_pspec = np.array(mth_pspec)
    mean_mthly_pspec = mthly_pspec.mean(axis=0)
    return kstep,mean_mthly_pspec

In [5]:
def compute_uspec(case,box):
    
    if box == 'box1':
        imin=75
        imax=256
        jmin=98
        jmax=326
    if box == 'box2':
        imin=70
        imax=253
        jmin=324
        jmax=568
    if box == 'box3':
        imin=60
        imax=245
        jmin=563
        jmax=826
    if box == 'box4':
        imin=21
        imax=208
        jmin=1089
        jmax=1391

    param='Uspec'
    varname = 'vozocrtx'
    WaveSpecResult = '/scratch/cnt0024/hmg2840/albert7a/EU36-spec/EU36-'+str(case)+'-'+str(param)+'/'
    !mkdir -p $WaveSpecResult
    YrMth  = ['2012m01','2012m02','2012m03','2012m04','2012m05','2012m06','2012m07','2012m08','2012m09','2012m10','2012m11','2012m12']

    for ii in np.arange(12):
        Mth = YrMth[ii]
        if not os.path.exists(WaveSpecResult+'WaveSpec_'+str(box)+'_'+param+'_'+Mth+'.npz'):
            print(WaveSpecResult+'WaveSpec_'+str(box)+'_'+param+'_'+Mth+'.npz')
            filename='/store/colombo/EU36/EU36-'+str(case)+'-S/1d/2012/EU36-'+str(case)+'_y'+str(YrMth[ii])+'d??.1d_gridU.nc'
            ds = xr.open_mfdataset(filename,chunks={'x':500,'y':500,'time_counter':1})
            data = ds[varname]
            if len(ds.nav_lon.dims) == 3:
                navlon=ds.nav_lon[0,imin:imax+1,jmin:jmax+1]
                navlat=ds.nav_lat[0,imin:imax+1,jmin:jmax+1]
            if len(ds.nav_lon.dims) == 2:
                navlon=ds.nav_lon[imin:imax+1,jmin:jmax+1]
                navlat=ds.nav_lat[imin:imax+1,jmin:jmax+1]
            compute_spec_for_box(imin,imax,jmin,jmax,data,Mth,navlon,navlat,box,WaveSpecResult,param)


In [6]:
def compute_vspec(case,box):
    
    if box == 'box1':
        imin=75
        imax=256
        jmin=98
        jmax=326
    if box == 'box2':
        imin=70
        imax=253
        jmin=324
        jmax=568
    if box == 'box3':
        imin=60
        imax=245
        jmin=563
        jmax=826
    if box == 'box4':
        imin=21
        imax=208
        jmin=1089
        jmax=1391

    param='Vspec'
    varname = 'vomecrty'
    WaveSpecResult = '/scratch/cnt0024/hmg2840/albert7a/EU36-spec/EU36-'+str(case)+'-'+str(param)+'/'
    !mkdir -p $WaveSpecResult
    YrMth  = ['2012m01','2012m02','2012m03','2012m04','2012m05','2012m06','2012m07','2012m08','2012m09','2012m10','2012m11','2012m12']

    for ii in np.arange(12):
        Mth = YrMth[ii]
        if not os.path.exists(WaveSpecResult+'WaveSpec_'+str(box)+'_'+param+'_'+Mth+'.npz'):
            print(WaveSpecResult+'WaveSpec_'+str(box)+'_'+param+'_'+Mth+'.npz')
            filename='/store/colombo/EU36/EU36-'+str(case)+'-S/1d/2012/EU36-'+str(case)+'_y'+str(YrMth[ii])+'d??.1d_gridV.nc'
            ds = xr.open_mfdataset(filename,chunks={'x':500,'y':500,'time_counter':1})
            data = ds[varname]
            if len(ds.nav_lon.dims) == 3:
                navlon=ds.nav_lon[0,imin:imax+1,jmin:jmax+1]
                navlat=ds.nav_lat[0,imin:imax+1,jmin:jmax+1]
            if len(ds.nav_lon.dims) == 2:
                navlon=ds.nav_lon[imin:imax+1,jmin:jmax+1]
                navlat=ds.nav_lat[imin:imax+1,jmin:jmax+1]
            compute_spec_for_box(imin,imax,jmin,jmax,data,Mth,navlon,navlat,box,WaveSpecResult,param)


In [7]:
def compute_kespec(case,box):
    YrMth  = ['2012m01','2012m02','2012m03','2012m04','2012m05','2012m06','2012m07','2012m08','2012m09','2012m10','2012m11','2012m12']
    # Folders containing U and V spectral
    u_database = '/scratch/cnt0024/hmg2840/albert7a/EU36-spec/EU36-'+str(case)+'-Uspec/'
    v_database = '/scratch/cnt0024/hmg2840/albert7a/EU36-spec/EU36-'+str(case)+'-Vspec/'
    # Folders to contain computed KE spectral
    KEspecFolder = '/scratch/cnt0024/hmg2840/albert7a/EU36-spec/EU36-'+str(case)+'-KEspec/'
    !mkdir -p $KEspecFolder
    u_filenames = sorted(glob.glob(u_database + 'WaveSpec_'+str(box)+'_Uspec_*.npz'))
    v_filenames = sorted(glob.glob(v_database + 'WaveSpec_'+str(box)+'_Vspec_*.npz'))
    for i in range(len(u_filenames)):
        if not os.path.exists(KEspecFolder+'WaveSpec_'+str(box)+'_KEspec_'+YrMth[i]+'.npz'):
            uspec = np.load(u_filenames[i])['pspec']
            vspec = np.load(v_filenames[i])['pspec']
            kspec = np.load(u_filenames[i])['kspec']
            KEspec = 0.5*(uspec + vspec)
            print(KEspecFolder+'WaveSpec_'+str(box)+'_KEspec_'+YrMth[i]+'.npz')
            np.savez(KEspecFolder+'WaveSpec_'+str(box)+'_KEspec_'+YrMth[i]+'.npz',kspec = kspec,KEspec = KEspec)

In [8]:
box='box3'
case='MPC001'

In [9]:
    if box == 'box3':
        imin=60
        imax=245
        jmin=563
        jmax=826
    if box == 'box4':
        imin=21
        imax=208
        jmin=1089
        jmax=1391

    param='Uspec'
    varname = 'vozocrtx'
    WaveSpecResult = '/scratch/cnt0024/hmg2840/albert7a/EU36-spec/EU36-'+str(case)+'-'+str(param)+'/'
    !mkdir -p $WaveSpecResult
    YrMth  = ['2012m01','2012m02','2012m03','2012m04','2012m05','2012m06','2012m07','2012m08','2012m09','2012m10','2012m11','2012m12']



In [10]:
        ii=0
        Mth = YrMth[ii]


In [11]:
            filename='/store/colombo/EU36/EU36-'+str(case)+'-S/1d/2012/EU36-'+str(case)+'_y'+str(YrMth[ii])+'d??.1d_gridU.nc'
            ds = xr.open_mfdataset(filename,chunks={'x':500,'y':500,'time_counter':1})
            data = ds[varname]
            if len(ds.nav_lon.dims) == 3:
                navlon=ds.nav_lon[0,imin:imax+1,jmin:jmax+1]
                navlat=ds.nav_lat[0,imin:imax+1,jmin:jmax+1]
            if len(ds.nav_lon.dims) == 2:
                navlon=ds.nav_lon[imin:imax+1,jmin:jmax+1]
                navlat=ds.nav_lat[imin:imax+1,jmin:jmax+1]


In [12]:
    var = get_values_in_box(imin,imax,jmin,jmax,data)


In [13]:
    days = len(var)
    mth_pspec = []


In [15]:
datab.shape

(264, 186)

In [16]:
navlon.shape

(186, 169)

In [17]:
navlat.shape

(186, 169)

In [20]:
ds.nav_lat

<xarray.DataArray 'nav_lat' (y: 1718, x: 732)>
dask.array<where, shape=(1718, 732), dtype=float32, chunksize=(500, 500)>
Coordinates:
    nav_lat  (y, x) float32 dask.array<chunksize=(500, 500), meta=np.ndarray>
    nav_lon  (y, x) float32 dask.array<chunksize=(500, 500), meta=np.ndarray>
Dimensions without coordinates: y, x
Attributes:
    standard_name:  latitude
    long_name:      Latitude
    units:          degrees_north

In [21]:
len(ds.nav_lon.dims)

2

In [22]:
print(imin,imax,jmin,jmax)

60 245 563 826


In [25]:
                navlon=ds.nav_lon[jmin:jmax+1,imin:imax+1]
                navlat=ds.nav_lat[jmin:jmax+1,imin:imax+1]


In [26]:
navlon.shape

(264, 186)

In [14]:
        it=0
        arr = var[it]
        datab = arr.squeeze()
        pspec,kstep = pp.wavenumber_spectra(datab,navlon,navlat)


ZeroDivisionError: float division by zero